In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import sys
import asyncio
import pandas as pd
from datetime import datetime
from uuid import UUID

# Add parent directory to path to import from back/
sys.path.append("..")
from back.search import SearchAssistant
from back.qdrant_search import QdrantSearchEngine

In [84]:
import nest_asyncio
nest_asyncio.apply()

In [85]:
from back.qdrant_search import QdrantSearchEngine

In [86]:
self = QdrantSearchEngine()

In [87]:
from vexa import VexaAPI

vexa = VexaAPI()
await vexa.get_user_info()



Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


{'id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'email': '2280905@gmail.com',
 'is_allowed_send_init_message': True,
 'username': 'string',
 'first_name': 'string',
 'last_name': 'string',
 'image': None,
 'created_timestamp': '2024-05-28T11:16:50.894711Z',
 'updated_timestamp': '2024-11-05T12:53:18.974930Z'}

In [88]:
from qdrant_search import QdrantSearchEngine

In [104]:
from psql_helpers import get_meetings_by_user_id
meetings, total = await get_meetings_by_user_id(vexa.user_id)
meeting_ids = [str(meeting['meeting_id']) for meeting in meetings]



In [112]:
q = 'Dmitry'

In [115]:
self = QdrantSearchEngine()

In [116]:
await self.search(q, meeting_ids, 20000, 0.00003)

Error in search: 1 validation error for SearchRequest
search_params
  Extra inputs are not permitted [type=extra_forbidden, input_value=SearchParams(hnsw_ef=128,...one, indexed_only=False), input_type=SearchParams]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden


[]

In [111]:
info = await self.client.get_collection(self.collection_name)
info.points_count


24954

In [91]:
import pandas as pd

In [92]:
pd.options.display.max_colwidth = 1000

In [94]:
meeting_ids = [str(meeting['meeting_id']) for meeting in meetings]



In [95]:
from qdrant_client import models

In [96]:
limit =200000

In [97]:
min_score = 0.00003

In [98]:
self.weights.items()

dict_items([('topic_name', 1.0), ('summary', 0.7), ('details', 0.5), ('speaker', 0.8), ('transcript', 0.6)])

In [99]:
query_text = 'Dmitry'
#meeting_ids = ['553fcfe6-0143-4971-99e7-213245f63347']

In [66]:
all_results = []

# Search each field type separately
for field_type, weight in self.weights.items():
    must_conditions = [
        models.FieldCondition(
            key="vector_type",
            match=models.MatchValue(value=field_type)
        ),
        models.FieldCondition(
            key="meeting_id",
            match=models.MatchAny(any=meeting_ids)
        )
    ]

    search_filter = models.Filter(must=must_conditions)

    results = await self.client.query(
        collection_name=self.collection_name,
        query_text=query_text,
        query_filter=search_filter,
        limit=limit * 2,
        score_threshold=min_score
    )

    for hit in results:
        hit.score *= weight
        # Properly structure the data
        result_dict = {
            'id': hit.id,
            'document': hit.document,
            'score': hit.score,
            'vector_type': hit.metadata['vector_type']  # This is needed for pivot
        }
        # Add all metadata fields
        result_dict.update(hit.metadata)
        all_results.append(result_dict)

In [67]:
df = pd.DataFrame(all_results)

In [68]:
metadata_agg = df.groupby('id').agg({
    'topic_name': 'first',
    'summary': 'first',
    'details': 'first',
    'topic_type': 'first',
    'timestamp': 'first',
    'meeting_id': 'first',
})
metadata_agg= metadata_agg.join(df[['id','speaker_name','score','discussion_id']].groupby('id').max()).sort_values('score', ascending=False)

In [69]:
metadata_agg.groupby('discussion_id').agg({
    'topic_name': 'first',
    'summary': 'first',
    'details': 'first',
    'topic_type': 'first',
    'timestamp': 'first',
    'meeting_id': 'first',
    'score': 'max'
}).sort_values('score', ascending=False)

,topic_name,summary,details,topic_type,timestamp,meeting_id,score
discussion_id,,,,,,,
7d7f9ab8-3d69-48e4-8266-feddc752e6bf,Dmitry Grankin,A participant in the meeting who engages with Ethan Noah about the marketing strategies of AI Marketing Directory.,He asks questions about the effectiveness of the marketing strategies and the targeting of specific audiences.,person,2024-10-10T18:45:35.984000,b8880903-8d2c-4924-b6a8-64f73b8b52ad,0.509804
e14323e3-94a0-4bab-9de2-d7c4d746d81f,Nikita,Nikita is mentioned as a candidate who struggled to answer questions about scaling and user management.,Sergey notes that Nikita provided vague responses and lacked a clear understanding of the technical requirements.,person,2024-09-18T11:57:28.220000,52cd3cf1-7c60-4e77-b861-69ca6ffe914e,0.500000
e5128009-eeb1-4d23-bdd8-4ef75bb69bba,Collaboration with Dmitry,Engage Dmitry to assist with prompt engineering and AI content generation.,Dmitry has offered to provide a couple of hours of support to help refine the AI's prompt engineering and improve the output quality. A follow-up meeting is to be scheduled to discuss this collaboration further.,action plan,2024-10-08T10:33:20.200000,cdf9078d-87e0-49cb-9b62-71596961f436,0.427778
eb7d6752-f25a-46bb-bfaf-ef8392d5c1bc,PDF to Docs,PDF to Docs is a conversion tool mentioned in the meeting that converts PDF files to Google Docs format.,Dmitry discusses the launch of this tool as part of their product offerings.,product,2024-09-25T13:28:55.560000,0aee5d35-5e04-4b48-b428-6e2979aab55c,0.416667
d637d24b-f7cb-4e2b-b99c-e433e08c0ac0,User acquisition and retention metrics,"DG shared positive metrics regarding user acquisition and retention, indicating growth and engagement with the product.","DG reported having 15 active users who have used the product at least three times in the last week, with a retention rate that shows promise for future growth.",update,2024-10-24T09:32:32.980000,50d5d659-3a57-42f2-8f32-31c7145ebe91,0.400967
...,...,...,...,...,...,...,...
23647e2d-0cfe-4dd4-9716-878efa66bf47,Surfer SEO,Surfer SEO is a tool that helps optimize content for better search engine rankings.,Rick mentions Surfer SEO as a valuable tool for creating content clusters and improving SEO strategies.,product,2024-10-03T18:06:40.070000,6cf0b989-5e9c-4a57-a058-8f6ac0ca8025,0.000737
d1b075dc-4c5a-4540-af86-5631b69dcc03,Surfer,Surfer is an SEO tool mentioned by Rick Tousseyn for optimizing content based on search queries.,"Rick uses Surfer to benchmark content and optimize articles, discussing its features and how it compares to other tools.",product,2024-10-01T09:19:05.583999,bdc05628-4278-4cda-9395-49906c348d7d,0.000732
d9f57765-8201-45d4-ab70-5daaa85fa4e3,Discuss collaboration on legal tech innovations,"The team discussed potential collaborations on legal tech innovations, particularly in the area of AI and automation.",There is a strong interest in exploring partnerships with other legal tech firms to enhance their service offerings. The team aims to stay updated on developments in the legal tech space and identify potential collaborators.,future projection,2024-10-22T17:44:20.910000,5824bf7d-b528-429f-90fe-7048fa932539,0.000728


In [70]:
metadata_agg.sort_values('score', ascending=False)


,topic_name,summary,details,topic_type,timestamp,meeting_id,speaker_name,score,discussion_id
id,,,,,,,,,
37c66b9d-1229-4630-9aef-84cf7b646379,Dmitry Grankin,None,None,person,2024-10-10T18:45:35.984000,b8880903-8d2c-4924-b6a8-64f73b8b52ad,Dmitry Grankin,0.509804,7d7f9ab8-3d69-48e4-8266-feddc752e6bf
4e7be12c-8850-4fd7-b76e-36f22d812a62,Nikita,None,None,person,2024-09-18T11:57:28.220000,52cd3cf1-7c60-4e77-b861-69ca6ffe914e,Sergey Ryabenko,0.500000,e14323e3-94a0-4bab-9de2-d7c4d746d81f
7f6b1b03-f59e-4d24-a4fb-1b67e00e50e3,None,Engage Dmitry to assist with prompt engineering and AI content generation.,None,action plan,2024-10-08T10:33:20.200000,cdf9078d-87e0-49cb-9b62-71596961f436,David Sterry,0.427778,e5128009-eeb1-4d23-bdd8-4ef75bb69bba
3d01d12a-c173-4f0b-a5e2-1510c1c99c75,None,None,Dmitry discusses the launch of this tool as part of their product offerings.,product,2024-09-25T13:28:55.560000,0aee5d35-5e04-4b48-b428-6e2979aab55c,Дмитрий Гранкин,0.416667,eb7d6752-f25a-46bb-bfaf-ef8392d5c1bc
c966f2bc-bbcc-4514-b250-69bb1fe63f0c,None,None,None,update,2024-10-24T09:32:32.980000,50d5d659-3a57-42f2-8f32-31c7145ebe91,Dmitry Grankin,0.400967,d637d24b-f7cb-4e2b-b99c-e433e08c0ac0
...,...,...,...,...,...,...,...,...,...
b4340a65-02eb-4b75-ab14-715ff607a827,None,None,None,None,2024-10-10T18:59:25.568000Z,b8880903-8d2c-4924-b6a8-64f73b8b52ad,NaN,0.000224,NaN
c52f0f6d-eb9d-471b-bb04-ee1b92c52008,None,None,None,None,2024-09-20T20:18:14.656000Z,cac618cd-fb7f-4d15-ada4-0f06358b13e5,NaN,0.000224,NaN
305bf51d-8c5e-4702-b151-d683d0e7d65a,None,None,None,None,2024-10-03T17:55:21.580000Z,3be31222-1a97-4042-b613-e5b6234789a0,NaN,0.000224,NaN


In [71]:
df.groupby('discussion_id').agg({
    'topic_name': 'first',
    'summary': 'first',
    'details': 'first',
    'topic_type': 'first',
    'timestamp': 'first',
    'meeting_id': 'first',
})

,topic_name,summary,details,topic_type,timestamp,meeting_id
discussion_id,,,,,,
003f43fe-0f5f-4f53-911b-2b59bd142e7d,Helping doctors in the UAE,"Igor discussed his current project that assists doctors in preparing for licensing exams in the UAE, addressing a significant need in the region.","The project helps doctors from various countries prepare for exams and navigate the licensing process in the UAE. The business model includes online exam preparation, document assistance, and medical English training, with a monthly revenue of $150,000.",task,2024-09-27T12:34:52,c450ea95-dc3b-4bf8-9e47-1d4f7b99e6b7
00545c02-e2c7-4f5d-8033-73e739d0b490,Schedule a follow-up meeting to continue the discussion,A follow-up meeting is proposed to delve deeper into the project details and ensure all team members are aligned.,The follow-up meeting will be scheduled to discuss the important topics identified in the initial discussion. The exact date and time will be determined based on team availability.,action plan,2024-10-03T10:40:37.280000,d5c2911a-358d-4f5c-a863-0d8224ef4b7d
0078fd07-428b-481f-9da3-50c27f40a7bd,FastAPI,FastAPI is a modern web framework for building APIs with Python 3.6+ based on standard Python type hints.,"Nikolay's application is built using FastAPI, which he describes as handling requests and managing asynchronous operations effectively.",product,2024-09-26T11:01:10.210000,10a11f1d-f985-4f46-9a80-3639d9e68a2a
008a5099-3241-4b52-a02e-9062c92bac66,Дмитрий Гранкин,"Dmitry Grankin is the founder of the startup Векса, which focuses on transcription services for Google Meet.","He discusses the development of his product, marketing strategies, and challenges in acquiring users.",person,2024-09-25T13:28:55.560000,0aee5d35-5e04-4b48-b428-6e2979aab55c
0119b247-4644-4447-bc51-70b8d178e712,Olga Nemirovskaya,"Olga Nemirovskaya is a speaker in the meeting, collaborating with Dmitriy on email marketing strategies and user engagement.",She suggests using a simpler email marketing service than MailChimp and discusses the importance of targeted outreach to users.,person,2024-09-17T09:49:06.520000,f3dd4aba-acde-46e0-9b6c-12591c740e0e
...,...,...,...,...,...,...
ff7c957b-5c5e-492e-a475-f5ea87eacfb9,Telegram,Telegram is another messaging platform integrated into the application for sharing artifacts.,Users will be able to send artifacts through Telegram as part of the sharing options.,product,2024-09-18T19:04:07.590000,ea5b6130-fa81-4541-8c22-4846ad9bc194
ff8132bf-c231-48ad-b7b8-61b96a744e35,наук,"Reference to 'наук', which translates to 'science' or 'knowledge' in English.","The speaker noted that there is no information or data available regarding 'наук' today, suggesting a lack of updates or progress.",topic,2024-09-27T09:14:13.760000,03a64799-e044-47a3-851e-21b4819a53d4
ffaefe65-b0f4-4369-9f6b-86de5621f503,список звонков,"A list of calls that includes participants, date, and time, which is central to the application's functionality.","The list will be searchable and filterable, providing users with easy access to call information.",topic,2024-09-18T19:04:07.590000,ea5b6130-fa81-4541-8c22-4846ad9bc194


In [72]:
metadata_agg

,topic_name,summary,details,topic_type,timestamp,meeting_id,speaker_name,score,discussion_id
id,,,,,,,,,
37c66b9d-1229-4630-9aef-84cf7b646379,Dmitry Grankin,None,None,person,2024-10-10T18:45:35.984000,b8880903-8d2c-4924-b6a8-64f73b8b52ad,Dmitry Grankin,0.509804,7d7f9ab8-3d69-48e4-8266-feddc752e6bf
4e7be12c-8850-4fd7-b76e-36f22d812a62,Nikita,None,None,person,2024-09-18T11:57:28.220000,52cd3cf1-7c60-4e77-b861-69ca6ffe914e,Sergey Ryabenko,0.500000,e14323e3-94a0-4bab-9de2-d7c4d746d81f
7f6b1b03-f59e-4d24-a4fb-1b67e00e50e3,None,Engage Dmitry to assist with prompt engineering and AI content generation.,None,action plan,2024-10-08T10:33:20.200000,cdf9078d-87e0-49cb-9b62-71596961f436,David Sterry,0.427778,e5128009-eeb1-4d23-bdd8-4ef75bb69bba
3d01d12a-c173-4f0b-a5e2-1510c1c99c75,None,None,Dmitry discusses the launch of this tool as part of their product offerings.,product,2024-09-25T13:28:55.560000,0aee5d35-5e04-4b48-b428-6e2979aab55c,Дмитрий Гранкин,0.416667,eb7d6752-f25a-46bb-bfaf-ef8392d5c1bc
c966f2bc-bbcc-4514-b250-69bb1fe63f0c,None,None,None,update,2024-10-24T09:32:32.980000,50d5d659-3a57-42f2-8f32-31c7145ebe91,Dmitry Grankin,0.400967,d637d24b-f7cb-4e2b-b99c-e433e08c0ac0
...,...,...,...,...,...,...,...,...,...
b4340a65-02eb-4b75-ab14-715ff607a827,None,None,None,None,2024-10-10T18:59:25.568000Z,b8880903-8d2c-4924-b6a8-64f73b8b52ad,NaN,0.000224,NaN
c52f0f6d-eb9d-471b-bb04-ee1b92c52008,None,None,None,None,2024-09-20T20:18:14.656000Z,cac618cd-fb7f-4d15-ada4-0f06358b13e5,NaN,0.000224,NaN
305bf51d-8c5e-4702-b151-d683d0e7d65a,None,None,None,None,2024-10-03T17:55:21.580000Z,3be31222-1a97-4042-b613-e5b6234789a0,NaN,0.000224,NaN


In [73]:
df[['discussion_id','speaker_name','score']].groupby('discussion_id').max()

,speaker_name,score
discussion_id,,
003f43fe-0f5f-4f53-911b-2b59bd142e7d,Igor Vinidiktov,0.004000
00545c02-e2c7-4f5d-8033-73e739d0b490,nan (1),0.001094
0078fd07-428b-481f-9da3-50c27f40a7bd,Nikolay,0.088889
008a5099-3241-4b52-a02e-9062c92bac66,Дмитрий Гранкин,0.058824
0119b247-4644-4447-bc51-70b8d178e712,Olga Nemirovskaya,0.010000
...,...,...
ff7c957b-5c5e-492e-a475-f5ea87eacfb9,Dmitriy Grankin,0.012500
ff8132bf-c231-48ad-b7b8-61b96a744e35,nan,0.003774
ffaefe65-b0f4-4369-9f6b-86de5621f503,Dmitriy Grankin,0.009195


In [74]:
[['topic_name','summary','']]

[['topic_name', 'summary', '']]

In [75]:
df.head()

,id,document,score,vector_type,meeting_id,speaker_name,topic_type,discussion_id,timestamp,vector_source,topic_name,summary,details,content,speaker,indices,chunk_index
0,37c66b9d-1229-4630-9aef-84cf7b646379,Dmitry Grankin,0.509804,topic_name,b8880903-8d2c-4924-b6a8-64f73b8b52ad,Dmitry Grankin,person,7d7f9ab8-3d69-48e4-8266-feddc752e6bf,2024-10-10T18:45:35.984000,discussion,Dmitry Grankin,NaN,NaN,NaN,NaN,NaN,NaN
1,4e7be12c-8850-4fd7-b76e-36f22d812a62,Nikita,0.500000,topic_name,52cd3cf1-7c60-4e77-b861-69ca6ffe914e,Sergey Ryabenko,person,e14323e3-94a0-4bab-9de2-d7c4d746d81f,2024-09-18T11:57:28.220000,discussion,Nikita,NaN,NaN,NaN,NaN,NaN,NaN
2,af995c89-d281-4101-8ca3-d1578d09e62f,Dmitry Grankin,0.343333,topic_name,7343734f-7311-4844-aa5e-86f1f93595c5,Dmitry Grankin,person,be15cb1a-95c5-44a8-8463-743058995e42,2024-10-25T12:01:02.055999,discussion,Dmitry Grankin,NaN,NaN,NaN,NaN,NaN,NaN
3,12d6bdd4-4cb4-4711-a010-5eaf7985769d,Nikolay,0.333333,topic_name,ba6ca959-deb2-4757-b9be-217c6a636f4a,Nikolay,person,0bcf66ff-d334-47d2-96d5-f3a135b8c39f,2024-10-07T15:43:33.100000,discussion,Nikolay,NaN,NaN,NaN,NaN,NaN,NaN
4,47850657-2641-47d3-9eed-4d35eadfe5dc,Dmitry Grankin,0.259259,topic_name,cdf9078d-87e0-49cb-9b62-71596961f436,Dmitry Grankin,person,c142cd5b-6534-4274-a33e-2f60dbf1c25f,2024-10-08T10:33:20.200000,discussion,Dmitry Grankin,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df['vector_type'].unique()

array(['topic_name', 'summary', 'details', 'speaker', 'transcript'],
      dtype=object)

In [77]:
df = await self.search_transcripts(query_text, meeting_ids,20000, 0.3)
df

,content,speaker,timestamp,similarity,meeting_id,indices,chunk_index
1,"— Здравствуйте, Дмитрий. — Здравствуйте, Игорь. How is it going? It's going very well. Как дела? Да ничего.",Igor Bessonov,2024-10-10T16:34:40.500000Z,0.597694,dd65ab42-1870-4158-9e2a-276fa5bda674,"[0, 1, 2, 3, 4]",0
2,"Eu já estou fazendo mais testes ainda para poder dizer um pouco melhor, mas eu estou curtindo muito o pouco que eu testei até agora, sabe? É, o Dmitry... A gente falou aqui, ó... Ah, vou ter que pôr aqui, porque assim, eu tô chique, né?",TBD,2024-09-24T16:03:04.100000Z,0.562857,1c084be2-5c19-4b05-95d7-b337219acf75,"[295, 296, 297, 298, 299]",59
0,"Yeah. Finish this pet is run through. So up here you've got these was my mask on. Up here you've got these four areas, users, organizations, projects, reporting and under each of those are the business processes that need to be contained within the training course. So invite users, verify users, adding users in manually, bulk That list has actually changed slightly, but what I'm trying to get across to you is that there are these 11 defined business processes. It's like a wider question that Dmitry asked and so then I think you know we are trying to build products that UI and UX friendly like me and Dmitry.",Nadya Gorodetskaya,2024-10-10T15:23:52.659999Z,0.500754,27959f97-ea15-48cb-87c8-88a5dce99da9,"[35, 36, 37, 38, 39]",7
3,"Подожди, а какой вопрос задать? Что-то у меня вообще как бы не это. Вот давай так, who is Дмитрий Гранкин? Короче, говорит, типа, недостаточно контекста, недостаточно контекста. Достаточно контекста. There is no information about Dmitry Grankin in the provided search results. Он мне просто, ну, говорит, типа, я понятия не имею, о чем ты говоришь.",Sergey Ryabenko,2024-10-20T13:23:50.974000Z,0.447619,aa87b543-f5c9-426c-a6ed-77b26f93e0da,"[255, 256, 257, 258, 259]",51
4,"Угу. Угу. Ну, вот. Один. Это вот Андрей ключ.",Sergey Ryabenko,2024-10-07T15:55:12.432000Z,0.395493,ba6ca959-deb2-4757-b9be-217c6a636f4a,"[170, 171, 172, 173, 174]",34
5,"hear you yes am i right assuming that what you're doing is like abridging global brands with the brazilian market yes yes exactly interesting um so i i'm dmitry i'm a founder of excel i'm i'm pretty sure you know something at least something about the company of course yeah We've been friends, we've been connected on LinkedIn now for a while, been following your posts. Yeah, so today is a special day as there is a product of cooperation. Yeah, I'm just really interested to ask a few questions about Brazilian market in general. What is it about?",Dmitriy Grankin,2024-09-26T17:05:46.408000Z,0.313259,9178c8e7-da6a-4883-9fe1-cc4ee30125ea,"[25, 26, 27, 28, 29]",5


In [78]:
r = await self.search_transcripts_with_context(query_text, meeting_ids, limit=20, min_score=0.5)

In [79]:
from psql_helpers import get_session
from psql_models import Meeting
from sqlalchemy import select

In [80]:
async with get_session() as session:
    query = select(Meeting).where(
        Meeting.meeting_id.in_([mid for mid in meeting_ids])
    )
    result = await session.execute(query)
    meetings = result.scalars().all()
meetings
meeting = meetings[0]
trans_df = pd.DataFrame(eval(meeting.transcript))
trans_df

,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp
0,TBD,TBD,"Тестирую работы системы end-to-end, как у меня тут вообще что-нибудь работает или ничего не работает, или поднялось уже.",None,None,[],2024-11-01T12:29:15.130000Z


In [81]:
from typing import List